In [1]:
import sys
sys.path.append("..")

This notebook will give an example how to use Renormalizer to calculate the ground state energy of the open boundary spin 1/2 1-D Heisenberg model.

$H = J \sum_i [S_i^z S_{i+1}^z + \frac{1}{2}(S_i^+ S_{i+1}^- + S_i^- S_{i+1}^+)]$

exact result (Bethe Anstatz):

L	     E/J

16	-6.9117371455749

24	-10.4537857604096

32	-13.9973156182243

48	-21.0859563143863

64	-28.1754248597421


In [2]:
from renormalizer.mps import Mps, Mpo, gs
from renormalizer.model import Model, Op
from renormalizer.model.basis import BasisHalfSpin

# define the # of spins
nspin = 32

# define the model
# sigma^+ = S^+
# sigma^- = S^-
# 1/2 sigma^x,y,z = S^x,y,z

ham_terms = []
for ispin in range(nspin-1):
    op1 = Op("sigma_z sigma_z", [ispin, ispin+1], 1.0/4)
    op2 = Op("sigma_+ sigma_-", [ispin, ispin+1], 1.0/2)
    op3 = Op("sigma_- sigma_+", [ispin, ispin+1], 1.0/2)
    ham_terms.extend([op1, op2, op3])

# set the spin order and local basis
basis = [BasisHalfSpin(i) for i in range(nspin)]

# construct Hamiltonian MPO
model = Model(basis, ham_terms)
mpo = Mpo(model)
print(f"mpo_bond_dims:{mpo.bond_dims}")

# set the sweep paramter
M=30
procedure = [[M, 0.2], [M, 0], [M, 0], [M,0], [M,0]]

# initialize a random MPS
qntot = 0
mps = Mps.random(model, qntot, M)

mps.optimize_config.procedure = procedure
mps.optimize_config.method = "2site"

# optimize MPS
energies, _ = gs.optimize_mps(mps.copy(), mpo)
print("gs energy:", min(energies))


2020-07-27 14:13:31,485[INFO] use numpy as backend
2020-07-27 14:13:31,488[INFO] use 64 bits
2020-07-27 14:13:31,711[DEBUG] # of operator terms: 93
2020-07-27 14:13:31,712[DEBUG] symbolic mpo algorithm: Hopcroft-Karp
2020-07-27 14:13:31,713[DEBUG] Input operator terms: 93
2020-07-27 14:13:31,752[DEBUG] After combination of the same terms: 93
2020-07-27 14:13:31,882[INFO] optimization method: 2site
2020-07-27 14:13:31,882[INFO] e_rtol: 1e-06
2020-07-27 14:13:31,883[INFO] e_atol: 1e-08
2020-07-27 14:13:31,898[DEBUG] isweep: 0
2020-07-27 14:13:31,898[DEBUG] mmax, percent: 30, 0.2
2020-07-27 14:13:31,899[DEBUG] current size: 376.2KiB, Matrix product bond dim:[1, 2, 4, 8, 16, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 1]
2020-07-27 14:13:31,900[DEBUG] optimize site: [30, 31]
2020-07-27 14:13:31,903[DEBUG] Hmat dim: 120
2020-07-27 14:13:31,904[DEBUG] use direct eigensolver
2020-07-27 14:13:31,911[DEBUG] energy: -1.4576135655612

mpo_bond_dims:[1, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 4, 1]


2020-07-27 14:13:32,332[DEBUG] energy: -2.7810557573131502
2020-07-27 14:13:32,334[DEBUG] optimize site: [26, 27]
2020-07-27 14:13:32,336[DEBUG] Hmat dim: 1920
2020-07-27 14:13:32,384[DEBUG] use davidson, HC hops: 58
2020-07-27 14:13:32,385[DEBUG] energy: -3.2397918544886695
2020-07-27 14:13:32,388[DEBUG] optimize site: [25, 26]
2020-07-27 14:13:32,392[DEBUG] Hmat dim: 3600
2020-07-27 14:13:32,460[DEBUG] use davidson, HC hops: 64
2020-07-27 14:13:32,460[DEBUG] energy: -3.656106610372086
2020-07-27 14:13:32,464[DEBUG] optimize site: [24, 25]
2020-07-27 14:13:32,467[DEBUG] Hmat dim: 3600
2020-07-27 14:13:32,544[DEBUG] use davidson, HC hops: 62
2020-07-27 14:13:32,545[DEBUG] energy: -4.170555831462217
2020-07-27 14:13:32,548[DEBUG] optimize site: [23, 24]
2020-07-27 14:13:32,550[DEBUG] Hmat dim: 3600
2020-07-27 14:13:32,636[DEBUG] use davidson, HC hops: 69
2020-07-27 14:13:32,637[DEBUG] energy: -4.51479751259156
2020-07-27 14:13:32,640[DEBUG] optimize site: [22, 23]
2020-07-27 14:13:32,64

gs energy: -13.997315092466646
